In [ ]:
import pandas as pd
import os
import glob
from PIL import Image, ImageDraw
import numpy as np
import matplotlib.pyplot as plt
import random
from sklearn.model_selection import train_test_split
import shutil
import torch
from IPython.display import Image  # for displaying images
import os 
import random
import shutil
import PIL

In [ ]:
#!git clone https://github.com/WongKinYiu/yolov7

In [ ]:
# Reading the csv file
pothole_df = pd.read_csv('/Users/lufunomulaudzi/Downloads/TheDataset/train_df.csv')
pothole_df.head()

In [ ]:
classMap = {'pothole': 0, 'nopothole': 1}

In [ ]:
# Create the main data folder
#!mkdir potholeData
# Create images and labels data folders
#!mkdir potholeData/images
#!mkdir potholeData/labels
# Create train,val and test data folders for both images and labels
#!mkdir potholeData/images/train potholeData/images/val potholeData/images/test  potholeData/labels/train potholeData/labels/val potholeData/labels/test


In [ ]:
# Creating the list of images from the excel sheet
imgs = pothole_df['image_id'].unique().tolist()
# Loop through each of the image
for img in imgs:
    boundingDetails = []
    # First get the bounding box information for a particular image from the excel sheet
    boundingInfo = pothole_df.loc[pothole_df.image_id == img,:]
    # Loop through each row of the details
    for idx, row in boundingInfo.iterrows():
        # Get the class Id for the row
        class_id = 'pothole' if row["num_potholes"] > 0 else 'nopothole'
        # Convert the bounding box info into the format for YOLOV7
        # Get the width
        bb_width = row['w']
        # Get the height
        bb_height = row['h']
        # Get the centre coordinates
        bb_xcentre = (row['x'] + (row['x'] + row['w']))/2
        bb_ycentre = (row['y'] + (row['y'] + row['h']))/2
        # Normalise the coordinates by diving by width and height
        bb_xcentre /= row['w'] 
        bb_ycentre /= row['h'] 
        bb_width    /= row['w'] 
        bb_height   /= row['h']  
        # Append details in the list 
        boundingDetails.append("{} {:.3f} {:.3f} {:.3f} {:.3f}".format(class_id, bb_xcentre, bb_ycentre, bb_width, bb_height))
    # Create the file name to save this info     
    file_name = os.path.join("potholeData/labels", img.split(".")[0] + ".txt")
    # Save the annotation to disk
    print("\n".join(boundingDetails), file= open(file_name, "w"))

In [ ]:
# Get the list of all annotations
annotations = glob.glob('potholeData/labels' + '/*.txt')
annotations

In [ ]:
# Get the list of images from its folder
imagePath = '/Users/lufunomulaudzi/Downloads/TheDataset/Images'
images = glob.glob(imagePath + '/*.JPG')
images

In [ ]:
# Sort the annotations and images and the prepare the train ,test and validation sets
images.sort()
annotations.sort()

print(len(images))
print(len(annotations))
 
# Split the dataset into train-valid-test splits 
train_images, val_images, train_annotations, val_annotations = train_test_split(images, annotations, test_size = 0.2, random_state = 123)
val_images, test_images, val_annotations, test_annotations = train_test_split(val_images, val_annotations, test_size = 0.5, random_state = 123)


In [ ]:
#Utility function to copy images to destination folder
def move_files_to_folder(list_of_files, destination_folder):
    for f in list_of_files:
        try:
            shutil.copy(f, destination_folder)
        except:
            print(f)
            assert False

In [ ]:
# Copy the splits into the respective folders
move_files_to_folder(train_images, 'potholeData/images/train')
move_files_to_folder(val_images, 'potholeData/images/val/')
move_files_to_folder(test_images, 'potholeData/images/test/')
move_files_to_folder(train_annotations, 'potholeData/labels/train/')
move_files_to_folder(val_annotations, 'potholeData/labels/val/')
move_files_to_folder(test_annotations, 'potholeData/labels/test/')


In [20]:
!python yolov7/train.py --img 640 --cfg './yolov7/cfg/training/yolov7.yaml' --hyp data/hyp.scratch.p6.yaml --batch 4 --epochs 500 --data potholeData.yaml --weights yolov7m.pt --workers 4 --name yolo_pothole_det_m

YOLOR 🚀 v0.1-121-g2fdc7f1 torch 1.13.1 CPU

Namespace(weights='yolov7m.pt', cfg='./yolov7/cfg/training/yolov7.yaml', data='potholeData.yaml', hyp='./yolov7/data/hyp.scratch.p6.yaml', epochs=500, batch_size=4, img_size=[640, 640], rect=False, resume=False, nosave=False, notest=False, noautoanchor=False, evolve=False, bucket='', cache_images=False, image_weights=False, device='', multi_scale=False, single_cls=False, adam=False, sync_bn=False, local_rank=-1, workers=4, project='runs/train', entity=None, name='yolo_pothole_det_m', exist_ok=False, quad=False, linear_lr=False, label_smoothing=0.0, upload_dataset=False, bbox_interval=-1, save_period=-1, artifact_alias='latest', freeze=[0], v5_metric=False, world_size=1, global_rank=-1, save_dir='runs/train/yolo_pothole_det_m', total_batch_size=4)
tensorboard: Start with 'tensorboard --logdir runs/train', view at http://localhost:6006/
hyperparameters: lr0=0.01, lrf=0.2, momentum=0.937, weight_decay=0.0005, warmup_epochs=3.0, warmup_momentum=0